In [2]:
class S:
    pass
print(type(S))

<class 'type'>


In [ ]:
即 class S 的类型是 type,  type既可以返回一个对象的类型，也可以创建出新的类型

In [ ]:
下面用type (不用class) 就可以创建出类：

In [6]:
def fn(self,name='world'):
    print(f'Hello,{name}.')

Hello = type('Hello',(object,),dict(hello=fn))
h = Hello()
h.hello()
print(type(Hello))
print(type(h))

Hello,world.
<class 'type'>
<class '__main__.Hello'>


In [ ]:
Python解释器遇到class定义时，仅仅是扫描一下class定义的语法，然后调用type()函数创建出class。

In [ ]:
metaclass是Python面向对象里最难理解,以下内容看不懂也没关系，因为基本上你不会用到。

In [7]:
# 这个metaclass可以给我们自定义的MyList增加一个add方法：
class ListMetaclass(type):
    def __new__(cls,name,bases,attrs):
        attrs['add'] = lambda self,value: self.append(value)
        return type.__new__(cls,name,bases,attrs)

In [8]:
class Mylist(list,metaclass=ListMetaclass):
    pass

In [9]:
L = Mylist()
L.add(1)
L

[1]

In [ ]:
ORM框架

In [ ]:
' Simple ORM using metaclass '

class Field(object):

    def __init__(self, name, column_type):
        self.name = name
        self.column_type = column_type

    def __str__(self):
        return '<%s:%s>' % (self.__class__.__name__, self.name)

class StringField(Field):

    def __init__(self, name):
        super(StringField, self).__init__(name, 'varchar(100)')

class IntegerField(Field):

    def __init__(self, name):
        super(IntegerField, self).__init__(name, 'bigint')

class ModelMetaclass(type):

    def __new__(cls, name, bases, attrs):
        if name=='Model':
            return type.__new__(cls, name, bases, attrs)
        print('Found model: %s' % name)
        mappings = dict()
        for k, v in attrs.items():
            if isinstance(v, Field):
                print('Found mapping: %s ==> %s' % (k, v))
                mappings[k] = v
        for k in mappings.keys():
            attrs.pop(k)
        attrs['__mappings__'] = mappings # 保存属性和列的映射关系
        attrs['__table__'] = name # 假设表名和类名一致
        return type.__new__(cls, name, bases, attrs)

class Model(dict, metaclass=ModelMetaclass):

    def __init__(self, **kw):
        super(Model, self).__init__(**kw)

    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(r"'Model' object has no attribute '%s'" % key)

    def __setattr__(self, key, value):
        self[key] = value

    def save(self):
        fields = []
        params = []
        args = []
        for k, v in self.__mappings__.items():
            fields.append(v.name)
            params.append('?')
            args.append(getattr(self, k, None))
        sql = 'insert into %s (%s) values (%s)' % (self.__table__, ','.join(fields), ','.join(params))
        print('SQL: %s' % sql)
        print('ARGS: %s' % str(args))

# testing code:

class User(Model):
    id = IntegerField('id')
    name = StringField('username')
    email = StringField('email')
    password = StringField('password')

u = User(id=12345, name='Michael', email='test@orm.org', password='my-pwd')
u.save()